In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstration of Marlowe Runtime Capabilities

**Executive Summary**

This contract for a token sale uses nearly all of the features of the Marlowe Runtime backend. The `marlowe` commands `create`, `deposit`, `choose`, `notify`, and `advance` transition the contract, and the command `withdraw` redeems funds paid by the contract. It can be run using addresses for the parties, role tokens for the parties, or a mixture of addresses and role tokens for the parties.

**Highlights**

* Used `choose`, `deposit`, and `notify`.
* Includes a timeout.
* Uses a mix of addresses and role tokens for the parties.
* Involves native assets.
* Includes variations of transaction creation:
    * Multiple addresses fund a singletransaction.
    * One party receives the change from another party's transaction.
    * One party fund another's transaction.


**Parties**

* The *Offeror* is selling `Marlon` tokens.
* The *Purchaser* bids Ada for purchasing the `Marlon` tokens.
* The *Approver* has the option to block the transaction, perhaps because KYC requirements have not been met by the *Offeror* or *Purchaser*.


**Scenario**

1. The *Approver* creates the contract.
2. The *Offeror* chooses to offer 500 `Marlon` tokens.
3. The *Offeror* deposits the 500 `Marlon` tokens that the chose to offer.
4. The *Purchaser* bids 100 Ada for the `Marlon` tokens.
5. The *Offeror* accepts the bid made by the *Purchaser*.
6. The *Purchaser* deposits the 100 Ada that they bid.
7. The *Approver* declines to forbid the transaction by waiting for the next timeout in the contract, triggering the payment of the `Marlon` tokens to the *Purchaser*.
8. The contract is notified to close, paying the Ada to the *Offeror*.


![Token-Bid Contract](token-bid.png)

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

87b74aaa8662dd929ea4f953ad486c84d5866c9a


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Offeror

In [9]:
OFFEROR_SKEY="$TREASURY/john-fletcher.skey"
OFFEROR_VKEY="$TREASURY/john-fletcher.vkey"

Create the token offeror's keys, if necessary.

In [10]:
if [[ ! -e "$OFFEROR_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$OFFEROR_SKEY" --verification-key-file "$OFFEROR_VKEY"
fi
OFFEROR_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$OFFEROR_VKEY")
echo "$OFFEROR_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


#### The Purchaser

In [11]:
PURCHASER_SKEY="$TREASURY/thomas-kyd.skey"
PURCHASER_VKEY="$TREASURY/thomas-kyd.vkey"

Create the token purchaser's keys, if necessary.

In [12]:
if [[ ! -e "$PURCHASER_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PURCHASER_SKEY" --verification-key-file "$PURCHASER_VKEY"
fi
PURCHASER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PURCHASER_VKEY")
echo "$PURCHASER_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


#### The Approver

In [13]:
APPROVER_SKEY="$TREASURY/john-webster.skey"
APPROVER_VKEY="$TREASURY/john-webster.vkey"

Create the approvers's keys, if necessary.

In [14]:
if [[ ! -e "$APPROVER_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$APPROVER_SKEY" --verification-key-file "$APPROVER_VKEY"
fi
APPROVER_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$APPROVER_VKEY")
echo "$APPROVER_ADDR"

addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp


### Fund the addresses.

In [15]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 125000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$OFFEROR_ADDR" "$PURCHASER_ADDR" "$APPROVER_ADDR" \
  "$OFFEROR_ADDR" "$PURCHASER_ADDR" "$APPROVER_ADDR"  

TxId "c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8"


### Mint the tokens to be purchased

In [16]:
TOKEN_NAME=Marlon

In [17]:
POLICY_ID=$(
marlowe-cli util mint \
  --issuer "$FAUCET_ADDR:$FAUCET_SKEY" \
  --count 1000 \
  --out-file /dev/null \
  --submit 600 \
  "$TOKEN_NAME:$OFFEROR_ADDR" \
| sed -e 's/^PolicyID "\(.*\)"$/\1/' \
)
echo "POLICY_ID = $POLICY_ID"


Fee: Lovelace 188557
Size: 410 / 16384 = 2%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
POLICY_ID = 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d


In [18]:
TOKEN="$POLICY_ID.$TOKEN_NAME"
echo "TOKEN = $TOKEN"

TOKEN = 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.Marlon


### Time computations

In [19]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [20]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1667009451000


## The Contract

We set the parameters for the contract.

In [21]:
ADA=1000000

MINIMUM_ADA=$((3 * ADA))
MAXIMUM_TOKEN=1000
MAXIMUM_ADA=$((200 * ADA))

TOKEN_DEADLINE=$((NOW + 14 * MINUTE))
BID_DEADLINE=$((NOW + 15 * MINUTE))
ACCEPT_DEADLINE=$((NOW + 16 * MINUTE))
ADA_DEADLINE=$((NOW + 17 * MINUTE))
FORBID_DEADLINE=$((NOW + 18 * MINUTE))
NOTIFY_DEADLINE=$((NOW + 30 * MINUTE))

OFFEROR_PARTY="$OFFEROR_ADDR"
PURCHASER_PARTY="Purchaser"
#APPROVER_PARTY="Approver"
OFFEROR_PARTY_C="address: $OFFEROR_ADDR"
PURCHASER_PARTY_C="role_token: Purchaser"
APPROVER_PARTY_C="role_token: Approver"

Insert the parameters into the contract.

In [22]:
yaml2json << EOI > token-bid-1.contract
when:
- case:
    choose_between:
    - from: 1
      to: $MAXIMUM_TOKEN
    for_choice:
      choice_name: Offer
      choice_owner:
        $OFFEROR_PARTY_C
  then:
    when:
    - case:
        deposits:
          value_of_choice:
            choice_name: Offer
            choice_owner:
              $OFFEROR_PARTY_C
        into_account:
          $OFFEROR_PARTY_C
        of_token:
          currency_symbol: $POLICY_ID
          token_name: $TOKEN_NAME
        party:
          $OFFEROR_PARTY_C
      then:
        when:
        - case:
            choose_between:
            - from: 1
              to: $MAXIMUM_ADA
            for_choice:
              choice_name: Bid
              choice_owner:
                $PURCHASER_PARTY_C
          then:
            when:
            - case:
                choose_between:
                - from: 1
                  to: 1
                for_choice:
                  choice_name: Accept
                  choice_owner:
                    $OFFEROR_PARTY_C
              then:
                when:
                - case:
                    deposits:
                      value_of_choice:
                        choice_name: Bid
                        choice_owner:
                          $PURCHASER_PARTY_C
                    into_account:
                      $PURCHASER_PARTY_C
                    of_token:
                      currency_symbol: ''
                      token_name: ''
                    party:
                      $PURCHASER_PARTY_C
                  then:
                    when:
                    - case:
                        choose_between:
                        - from: 1
                          to: 1
                        for_choice:
                          choice_name: Forbid
                          choice_owner:
                            $APPROVER_PARTY_C
                      then: close
                    timeout: $FORBID_DEADLINE
                    timeout_continuation:
                      pay:
                        amount_of_token:
                          currency_symbol: ''
                          token_name: ''
                        in_account:
                          $PURCHASER_PARTY_C
                      from_account:
                        $PURCHASER_PARTY_C
                      token:
                        currency_symbol: ''
                        token_name: ''
                      to:
                        account:
                          $OFFEROR_PARTY_C
                      then:
                        pay:
                          amount_of_token:
                            currency_symbol: $POLICY_ID
                            token_name: $TOKEN_NAME
                          in_account:
                            $OFFEROR_PARTY_C
                        from_account:
                          $OFFEROR_PARTY_C
                        token:
                          currency_symbol: $POLICY_ID
                          token_name: $TOKEN_NAME
                        to:
                          party:
                            $PURCHASER_PARTY_C
                        then:
                          when:
                          - case:
                              notify_if: true
                            then: close
                          timeout: $NOTIFY_DEADLINE
                          timeout_continuation: close
                timeout: $ADA_DEADLINE
                timeout_continuation: close
            timeout: $ACCEPT_DEADLINE
            timeout_continuation: close
        timeout: $BID_DEADLINE
        timeout_continuation: close
    timeout: $TOKEN_DEADLINE
    timeout_continuation: close
timeout: $TOKEN_DEADLINE
timeout_continuation: close
EOI
cat token-bid-1.contract

{"timeout":1667010291000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1000}],"for_choice":{"choice_name":"Offer","choice_owner":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}}},"then":{"timeout":1667010291000,"timeout_continuation":"close","when":[{"case":{"deposits":{"value_of_choice":{"choice_name":"Offer","choice_owner":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}}},"into_account":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"},"of_token":{"currency_symbol":"8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d","token_name":"Marlon"},"party":{"address":"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"}},"then":{"timeout":1667010351000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":200000000}],"for_choice":{"choice_name":"Bid","choice_owner":{"role_token":"Purchaser"}}},"then":{"timeout":1667010411000,"

## Simulate the Contract

Simulate the contract before running it on chain.

We start the contract with an initial state of the minimum Ada.

In [23]:
yaml2json << EOI > token-bid-1.state
accounts:
- - - address: $APPROVER_ADDR
    - currency_symbol: ''
      token_name: ''
  - $MINIMUM_ADA
boundValues: []
choices: []
minTime: 1
EOI
cat token-bid-1.state

{"accounts":[[[{"address":"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp"},{"currency_symbol":"","token_name":""}],3000000]],"boundValues":[],"choices":[],"minTime":1}


### Transaction 1. Create the contract

In [24]:
marlowe-cli run initialize \
  --contract-file token-bid-1.contract \
  --state-file    token-bid-1.state \
  --out-file      token-bid-1.marlowe \
  --roles-currency "00000000000000000000000000000000000000000000000000000000" \
  --permanently-without-staking \
  --print-stats


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"

Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


### Transaction 2. The Offeror offers tokens

In [25]:
marlowe-cli run prepare \
  --marlowe-file token-bid-1.marlowe \
  --out-file     token-bid-2.marlowe \
  --choice-party "$OFFEROR_PARTY" \
  --choice-name Offer \
  --choice-number "$((MAXIMUM_TOKEN / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999}), txInputs = [NormalInput (IChoice (ChoiceId "Offer" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"") 500)]}

Datum size: 1041


### Transaction 3. The Offeror deposits the offered tokens

In [26]:
marlowe-cli run prepare \
  --marlowe-file token-bid-2.marlowe \
  --out-file     token-bid-3.marlowe \
  --deposit-account "$OFFEROR_PARTY" \
  --deposit-party "$OFFEROR_PARTY" \
  --deposit-amount "$((MAXIMUM_TOKEN / 2))" \
  --deposit-token "$TOKEN" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999}), txInputs = [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d" "Marlon") 500)]}

Datum size: 911


### Transaction 4. The Purchaser makes a bid

In [27]:
marlowe-cli run prepare \
  --marlowe-file token-bid-3.marlowe \
  --out-file     token-bid-4.marlowe \
  --choice-party "$PURCHASER_PARTY" \
  --choice-name Bid \
  --choice-number "$((MAXIMUM_ADA / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999}), txInputs = [NormalInput (IChoice (ChoiceId "Bid" "Purchaser") 100000000)]}

Datum size: 878


### Transaction 5. The Offeror accepts the Purchaser's bid

In [28]:
marlowe-cli run prepare \
  --marlowe-file token-bid-4.marlowe \
  --out-file     token-bid-5.marlowe \
  --choice-party "$OFFEROR_PARTY" \
  --choice-name Accept \
  --choice-number 1 \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999}), txInputs = [NormalInput (IChoice (ChoiceId "Accept" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"") 1)]}

Datum size: 845


### Transaction 6. The Purchaser deposits the Ada that they had bid

In [29]:
marlowe-cli run prepare \
  --marlowe-file token-bid-5.marlowe \
  --out-file     token-bid-6.marlowe \
  --deposit-account "$PURCHASER_PARTY" \
  --deposit-party "$PURCHASER_PARTY" \
  --deposit-amount "$((MAXIMUM_ADA / 2))" \
  --invalid-before    "$((NOW + 1 * MINUTE))" \
  --invalid-hereafter "$((NOW + 2 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667009511000},POSIXTime {getPOSIXTime = 1667009571999}), txInputs = [NormalInput (IDeposit "Purchaser" "Purchaser" (Token "" "") 100000000)]}

Datum size: 787


### Transaction 7. The Approver declines to forbid the sale by lettting the timeout pass

In [30]:
marlowe-cli run prepare \
  --marlowe-file token-bid-6.marlowe \
  --out-file     token-bid-7.marlowe \
  --invalid-before    "$((NOW + 20 * MINUTE))" \
  --invalid-hereafter "$((NOW + 21 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667010651000},POSIXTime {getPOSIXTime = 1667010711999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667010651000},POSIXTime {getPOSIXTime = 1667010711999}), txInputs = []}

Datum size: 363
Payment 1
  Acccount: "Purchaser"
  Payee: Account "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "Purchaser"
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."Marlon": 500


### Transaction 8. The contract is notified to make the final payment

In [31]:
marlowe-cli run prepare \
  --marlowe-file token-bid-7.marlowe \
  --out-file     token-bid-8.marlowe \
  --notify \
  --invalid-before    "$((NOW + 22 * MINUTE))" \
  --invalid-hereafter "$((NOW + 23 * MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1667010771000},POSIXTime {getPOSIXTime = 1667010831999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1667010771000},POSIXTime {getPOSIXTime = 1667010831999}), txInputs = [NormalInput INotify]}

Datum size: 207
Payment 1
  Acccount: "\"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp\""
  Payee: Party "\"addr_test1vp6g96pgt37le7lc0wew60shsdh2lmqrt57da9t6k64sltgasvxlp\""
  Ada: Lovelace {getLovelace = 3000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}


## Run the Contract

### Initial Funds

In [32]:
cardano-cli query utxo "${MAGIC[@]}" --address "$OFFEROR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     1        125000000 lovelace + TxOutDatumNone
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     4        125000000 lovelace + TxOutDatumNone
f188be7791ad0f42b3650aa2537ce3feb9ff498ac724d9eea1c199004fe48274     2        1043020 lovelace + 1000 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d61726c6f6e + TxOutDatumNone


In [33]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PURCHASER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     2        125000000 lovelace + TxOutDatumNone
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     5        125000000 lovelace + TxOutDatumNone


In [34]:
cardano-cli query utxo "${MAGIC[@]}" --address "$APPROVER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     3        125000000 lovelace + TxOutDatumNone
c836ce6c973a378e2d580e9050a3be4339866087b0f20cda2fe713be11ccb1a8     6        125000000 lovelace + TxOutDatumNone


### Transaction 1. Create the contract

Build the transaction.

In [35]:
TX_1=$(
marlowe create \
  --core-file token-bid-1.contract \
  --role="Purchaser=$PURCHASER_ADDR" \
  --role="Approver=$APPROVER_ADDR" \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$APPROVER_ADDR" \
  --manual-sign token-bid-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 8fbd62b7da52330297d5a6ebb013b636a28858712ffe44b6791df7c61ce82920#1


Sign the transaction.

In [36]:
cardano-cli transaction sign \
  --tx-body-file token-bid-1.txbody \
  --out-file     token-bid-1.tx \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [37]:
marlowe submit token-bid-1.tx

"{\"blockHeaderHash\":\"a0d1ef347cdd0b203e8647f262e775b0a0378688002a8c1feb073960f33c2121\",\"blockNo\":323906,\"slotNo\":7006328}"


### Transaction 2. The Offeror offers tokens

Build the transaction.

*Let's mix things up by spending the UTxOs of the offerer, but sending the change to the approver.*

In [38]:
TX_2=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$OFFEROR_PARTY" \
  --choice Offer \
  --value "$((MAXIMUM_TOKEN / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$APPROVER_ADDR" \
  --address "$OFFEROR_ADDR" \
  --manual-sign token-bid-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 1defc8b9cc186784b61de78be6f35a13a9f7c168b087da67babd79d0929b4e21


Sign the transaction.

In [39]:
cardano-cli transaction sign \
  --tx-body-file token-bid-2.txbody \
  --out-file     token-bid-2.tx \
  --signing-key-file "$OFFEROR_SKEY" \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [40]:
marlowe submit token-bid-2.tx

"{\"blockHeaderHash\":\"5280fcfcc89e6ae71215dc2580c7f94ca6cc4e24351975c187971385689bbcfc\",\"blockNo\":323907,\"slotNo\":7006343}"


### Transaction 3. The Offeror deposits the offered tokens

Build the transaction.

*Let's mix things up in a different way by having both the offeror and approver supply the funds.*

In [41]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --to-party "$OFFEROR_PARTY" \
  --from-party "$OFFEROR_PARTY" \
  --quantity "$((MAXIMUM_TOKEN / 2))" \
  --currency "$POLICY_ID" \
  --token-name "$TOKEN_NAME" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --address "$OFFEROR_ADDR" \
  --address "$APPROVER_ADDR" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = dfedb4cbd3873dae72b24fb71cb6812ab34e3196e8f3582f7dffaa571159ea71


Sign the transaction.

In [42]:
cardano-cli transaction sign \
  --tx-body-file token-bid-3.txbody \
  --out-file     token-bid-3.tx \
  --signing-key-file "$OFFEROR_SKEY" \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [43]:
marlowe submit token-bid-3.tx

"{\"blockHeaderHash\":\"ed9ced3d397951692a9a5fb4c623bd7c45901710fdafb01396e929cc9f8fa9a4\",\"blockNo\":323910,\"slotNo\":7006353}"


### Transaction 4. The Purchaser makes a bid

Build the transaction.

In [44]:
TX_4=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$PURCHASER_PARTY" \
  --choice Bid \
  --value "$((MAXIMUM_ADA / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = 5d12df6ebdd6d4ba2e13b1989a4d98a09fa1748c46c46d416cbdc7f632e8c0b0


Sign the transaction.

In [45]:
cardano-cli transaction sign \
  --tx-body-file token-bid-4.txbody \
  --out-file     token-bid-4.tx \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [46]:
marlowe submit token-bid-4.tx

"{\"blockHeaderHash\":\"0552d2786fa8953b23224e6219dcd28d3847e28eebb223d51b391170017227df\",\"blockNo\":323911,\"slotNo\":7006364}"


### Transaction 5. The Offeror accepts the Purchaser's bid

Build the transaction.

*Mix things up even more by having the purchaser pay for the transaction, but having the offeror also sign the transaction as evidence of their consent.*

In [47]:
TX_5=$(
marlowe choose \
  --contract "$CONTRACT_ID" \
  --party "$OFFEROR_PARTY" \
  --choice Accept \
  --value 1 \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-5.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = cbc0539f1211110353ef17231241dcc75af621d1991f3aa299510f1a398e9b9a


Sign the transaction.

In [48]:
cardano-cli transaction sign \
  --tx-body-file token-bid-5.txbody \
  --out-file     token-bid-5.tx \
  --signing-key-file "$OFFEROR_SKEY" \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [49]:
marlowe submit token-bid-5.tx

"{\"blockHeaderHash\":\"7c010cc9127773d051156c49bf180f0728e5c8a89a788179addf4f8ea350739d\",\"blockNo\":323912,\"slotNo\":7006374}"


### Transaction 6. The Purchaser deposits the Ada that they had bid

Build the transaction.

In [50]:
TX_6=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --to-party "$PURCHASER_PARTY" \
  --from-party "$PURCHASER_PARTY" \
  --lovelace "$((MAXIMUM_ADA / 2))" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-6.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_6 = $TX_6"

TX_6 = d38d1bd42ba4922b3c6bfb1bc74c9cfbea23ffb6f7b4866c85e76c29fc16557e


Sign the transaction.

In [51]:
cardano-cli transaction sign \
  --tx-body-file token-bid-6.txbody \
  --out-file     token-bid-6.tx \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [52]:
marlowe submit token-bid-6.tx

"{\"blockHeaderHash\":\"2de72062f6a623f150df6855514fd187ccf75cdef0a4d98c36f4434905148584\",\"blockNo\":323914,\"slotNo\":7006397}"


### Transaction 7. The Approver declines to forbid the sale by lettting the timeout pass

Wait fo the timeout, if necessary.

In [53]:
WAIT="$(((FORBID_DEADLINE - $(date -u +%s) * SECOND + 2 * MINUTE + 1 * SECOND) / SECOND))"
if [[ $WAIT -ge 0 ]]
then
  echo "Waiting $WAIT seconds before the timeout passes."
  sleep "$WAIT"
fi

Waiting 1053 seconds before the timeout passes.


Build the transaction.

*Mix this up by having the offeror pay, but the approver receive the change.*

In [54]:
TX_7=$(
marlowe advance \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 2 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$APPROVER_ADDR" \
  --address "$OFFEROR_ADDR" \
  --manual-sign token-bid-7.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_7 = $TX_7"

TX_7 = c1404f0a4ede22573654912fe7bbd5761e151a950775fb4f4756c16396068c37


Sign the transaction.

In [55]:
cardano-cli transaction sign \
  --tx-body-file token-bid-7.txbody \
  --out-file     token-bid-7.tx \
  --signing-key-file "$OFFEROR_SKEY" \
  --signing-key-file "$APPROVER_SKEY"

Submit the transaction using Marlowe Runtime.

In [56]:
marlowe submit token-bid-7.tx

"{\"blockHeaderHash\":\"66c64d92eb5f17c1af775b69d7358e14c527d814146937867cf5c9000dbab2df\",\"blockNo\":323964,\"slotNo\":7007479}"


### Transaction 8. The contract is notified to make the final payment

Build the transaction.

In [57]:
TX_8=$(
marlowe notify \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$(($(date -u +%s) * SECOND - 1 * MINUTE))" \
  --validity-upper-bound "$(($(date -u +%s) * SECOND + 2 * MINUTE))" \
  --change-address "$OFFEROR_ADDR" \
  --manual-sign token-bid-8.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_8 = $TX_8"

TX_8 = 57c6203f26f9f370341871ede5d243b5be378eb8f3f92bfbc7ad7995caf20c15


Sign the transaction.

In [58]:
cardano-cli transaction sign \
  --tx-body-file token-bid-8.txbody \
  --out-file     token-bid-8.tx \
  --signing-key-file "$OFFEROR_SKEY"

Submit the transaction using Marlowe Runtime.

In [59]:
marlowe submit token-bid-8.tx

"{\"blockHeaderHash\":\"ed6a03adf1d657f956873bb2105c86c5f6039759dfe33975ffd7b53e2cfdba82\",\"blockNo\":323965,\"slotNo\":7007484}"


### Transaction 9. Withdraw the tokens

In [60]:
TX_9=$(
marlowe withdraw \
  --contract "$CONTRACT_ID" \
  --role "$PURCHASER_PARTY" \
  --change-address "$PURCHASER_ADDR" \
  --manual-sign token-bid-9.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_9 = $TX_9"

TX_9 = ba6c141fe0fd41e4716e427e1b3961d90ef4609f3f1f19e4c2bc7d23631225d2


Sign the transaction.

In [61]:
cardano-cli transaction sign \
  --tx-body-file token-bid-9.txbody \
  --out-file     token-bid-9.tx \
  --signing-key-file "$PURCHASER_SKEY"

Submit the transaction using Marlowe Runtime.

In [62]:
marlowe submit token-bid-9.tx

"{\"blockHeaderHash\":\"aa40aa1686de33adb9ff6255526a8509c1e261bf6a3efbd7300a87a492d4a01d\",\"blockNo\":323966,\"slotNo\":7007530}"


### Final Funds

In [63]:
cardano-cli query utxo "${MAGIC[@]}" --address "$OFFEROR_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
57c6203f26f9f370341871ede5d243b5be378eb8f3f92bfbc7ad7995caf20c15     0        124464811 lovelace + TxOutDatumNone
57c6203f26f9f370341871ede5d243b5be378eb8f3f92bfbc7ad7995caf20c15     1        100000000 lovelace + TxOutDatumNone
dfedb4cbd3873dae72b24fb71cb6812ab34e3196e8f3582f7dffaa571159ea71     0        242961777 lovelace + TxOutDatumNone
dfedb4cbd3873dae72b24fb71cb6812ab34e3196e8f3582f7dffaa571159ea71     2        1043020 lovelace + 500 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d61726c6f6e + TxOutDatumNone


In [64]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PURCHASER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ba6c141fe0fd41e4716e427e1b3961d90ef4609f3f1f19e4c2bc7d23631225d2     0        123261338 lovelace + TxOutDatumNone
ba6c141fe0fd41e4716e427e1b3961d90ef4609f3f1f19e4c2bc7d23631225d2     1        1047330 lovelace + 1 6e3c8f57204763b2055d494cf844cd11be186775dd45b21f408f4076.507572636861736572 + TxOutDatumNone
ba6c141fe0fd41e4716e427e1b3961d90ef4609f3f1f19e4c2bc7d23631225d2     2        1043020 lovelace + 500 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d61726c6f6e + TxOutDatumNone
d38d1bd42ba4922b3c6bfb1bc74c9cfbea23ffb6f7b4866c85e76c29fc16557e     0        24163561 lovelace + TxOutDatumNone


In [65]:
cardano-cli query utxo "${MAGIC[@]}" --address "$APPROVER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
57c6203f26f9f370341871ede5d243b5be378eb8f3f92bfbc7ad7995caf20c15     2        3000000 lovelace + TxOutDatumNone
8fbd62b7da52330297d5a6ebb013b636a28858712ffe44b6791df7c61ce82920     2        1043020 lovelace + 1 6e3c8f57204763b2055d494cf844cd11be186775dd45b21f408f4076.417070726f766572 + TxOutDatumNone
c1404f0a4ede22573654912fe7bbd5761e151a950775fb4f4756c16396068c37     0        122928658 lovelace + TxOutDatumNone
